# Preparation
Dieses Notebook, dient lediglich zu Entwicklung von Vorverarbeitungsstrategien. Daher wird auf jedes Feature grob eingegangen.


## Bibliiotheken

In [3]:
import pandas as pd
import numpy as np

## Laden der Daten

In [4]:

labelList=['symboling','normalizedLosses','make','fuelType','aspiration','numOfDoors','bodyStyle','driveWheels','engineLocation',
           'wheelBase','length','width','height','curbWeight','engineType','numOfCylinders','engineSize','fuelSystem','bore',
           'stroke','compressionRatio','horsepower','peakRpm','cityMpg','highwayMpg','price']
df = pd.read_csv('../data/data_car-CopyForEDA.csv',delimiter=',',encoding='utf-8', names=labelList)
df = df.replace("?", np.nan)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
symboling           205 non-null int64
normalizedLosses    164 non-null float64
make                205 non-null object
fuelType            205 non-null object
aspiration          205 non-null object
numOfDoors          203 non-null object
bodyStyle           205 non-null object
driveWheels         205 non-null object
engineLocation      205 non-null object
wheelBase           205 non-null float64
length              205 non-null float64
width               205 non-null float64
height              205 non-null float64
curbWeight          205 non-null int64
engineType          205 non-null object
numOfCylinders      205 non-null object
engineSize          205 non-null int64
fuelSystem          205 non-null object
bore                201 non-null float64
stroke              201 non-null float64
compressionRatio    205 non-null float64
horsepower          203 non-null float64
peakRpm  

In [7]:
df = df.replace("?", np.nan)
df[['normalizedLosses']]=df[['normalizedLosses']].astype('float64')
df[['price']]=df[['price']].astype('float64')
df[['horsepower']]=df[['horsepower']].astype('float64')
df[['peakRpm']]=df[['peakRpm']].astype('float64')
df[['bore']]=df[['bore']].astype('float64')
df[['stroke']]=df[['stroke']].astype('float64')

In [13]:
df.sort_values(by='symboling', ascending=[False])

,symboling,normalizedLosses,make,fuelType,aspiration,numOfDoors,bodyStyle,driveWheels,engineLocation,wheelBase,...,engineSize,fuelSystem,bore,stroke,compressionRatio,horsepower,peakRpm,cityMpg,highwayMpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
29,3,145.0,dodge,gas,turbo,two,hatchback,fwd,front,95.9,...,156,mfi,3.60,3.90,7.0,145.0,5000.0,19,24,12964.0
178,3,197.0,toyota,gas,std,two,hatchback,rwd,front,102.9,...,171,mpfi,3.27,3.35,9.3,161.0,5200.0,20,24,16558.0
55,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,NaN,NaN,9.4,101.0,6000.0,17,23,10945.0
56,3,150.0,mazda,gas,std,two,hatchback,rwd,front,95.3,...,70,4bbl,NaN,NaN,9.4,101.0,6000.0,17,23,11845.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,-1,65.0,toyota,gas,std,four,hatchback,fwd,front,102.4,...,122,mpfi,3.31,3.54,8.7,92.0,4200.0,27,32,9988.0
204,-1,95.0,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114.0,5400.0,19,25,22625.0
194,-2,103.0,volvo,gas,std,four,sedan,rwd,front,104.3,...,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,12940.0
196,-2,103.0,volvo,gas,std,four,sedan,rwd,front,104.3,...,141,mpfi,3.78,3.15,9.5,114.0,5400.0,24,28,15985.0


## symboling
laut First-Touch kein Problem

## price
NAs werden duch den Druchschnittswert der Marke ausgewählt
Der Datentyp wird in int gewandelt

In [15]:
#df.price.isna().any()# > sind noch nas
df[['price']]=df[['price']].astype('float64')
for i in range(0, len(df.price),1):
    if(str(df.price[i]) == "nan"):
        nanmake = (df.make[i])
        subSetNanMake=df[df['make'] == nanmake]
        subSetNanMake = subSetNanMake.price.dropna()
        medianPerMake = subSetNanMake.median(axis = 0, skipna = True)
        df.price[i] = medianPerMake
#df.price.isna().any() => keine Nas Mehr

C:\Users\1810837475\.conda\envs\Kompensationsarbeit\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## numOfDoors
nas werden durch den Modus ersetzt der entsprechenden Wagenklasse(bodyStyle). ich geh davon aus dass eine wagenklasse meist die gleiche Türenanzhal hat => ich setzt den Modus dazu rein.

In [16]:
print(df.numOfDoors.unique())

['two' 'four' nan]


In [17]:
#df.numOfDoors.isna().any() #=> noch nas
for i in range(0, len(df.numOfDoors),1):
    if(str(df.numOfDoors[i]) == "nan"):
        nanBodyStyle = (df.bodyStyle[i])
        subSetNanBody=df[df['bodyStyle'] == nanBodyStyle]
        subSetNanBody = subSetNanBody.numOfDoors.dropna()
        modusNumOfDoors = subSetNanBody.mode().iloc[0]
        #print(modusNumOfDoors)
        df.numOfDoors[i] = modusNumOfDoors
#df.numOfDoors.isna().any() # => keine Nas Mehr

C:\Users\1810837475\.conda\envs\Kompensationsarbeit\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Der nachfolgede Teil zu numOfDoors ist aufgekommen, da im OHE ein außergewöhnlicher wert erschienen ist.

In [18]:
print(df.numOfDoors.unique())

['two' 'four']


# Bore & Stroke & horse power & Peakrpm
Das sind die Parameter, die den Hubraum(engine-Size) beschreiben. Nach ewig langer recherche errechne ich aber auch  bei Datensätzen bei welchen ich sowohl bore, stroke, horsepower & PRPM als auch engine habnciht den richtigen Hubrubraum. Somit geh ich anders vor.
    1) finde Nonvalues in den bereichen (glück = bei Stroke&bore als auch bei HP&PRPM sind es jeweils die gleichen Datensätze)
    2) wähle von nonvalue die Enigine size
    3) finde die nächst kleinere Engine size
    4) impute von den nächst kleineren die bore & Storke values dem leeren Feld

In [19]:
df.bore.isna().any()
df.stroke.isna().any()

True

In [20]:
# Bore &Stroke
#from operator import itemgetter
for i in range(0, len(df.stroke),1):
    if(str(df.stroke[i]) == "nan"):
        #print(i)
        refferenceEngine = df.engineSize[i]
        #print(refferenceEngine)
        #finde alle die kleiner sind
        possibleMachines = [] 
        for j in range(0, len(df),1):
            if(df.engineSize[j]<refferenceEngine):
                possibleMachines.append([j, df.engineSize[j], df.bore[j], df.stroke[j]])
            else:
                pass
        #print(possibleMachines)
        detectedValue=[possibleMachines[0]]# finde aus den möglichen Maschinen, die nächst kleinere
        for j in range(0, len(possibleMachines),1):
           
            if(int(detectedValue[0][1]) >= int(possibleMachines[j][1])):
                pass
            else:
                detectedValue = [possibleMachines[j]]
        print(detectedValue)
            
        #jetzt den Update im Dataframe mit den broke & stroke von übergebenen DetectedValue (bei index = 0)
        df.bore[i] = detectedValue[0][2]
        df.stroke[i] = detectedValue[0][3]
df[['bore']]=df[['bore']].astype('float64')
df[['stroke']]=df[['stroke']].astype('float64')

[[18, 61, '2.91', '3.03']]
[[18, 61, '2.91', '3.03']]
[[18, 61, '2.91', '3.03']]
[[32, 79, '2.91', '3.07']]


C:\Users\1810837475\.conda\envs\Kompensationsarbeit\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\1810837475\.conda\envs\Kompensationsarbeit\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
df.bore.isna().any()
df.stroke.isna().any()

False

In [22]:
# HP&PRPM
for i in range(0, len(df.horsepower),1):
    if(str(df.horsepower[i]) == "nan"):
        refferenceEngine = df.engineSize[i]
        possibleMachines = [] 
        for j in range(0, len(df),1):
            if(df.engineSize[j]<refferenceEngine):
                possibleMachines.append([j, df.engineSize[j], df.horsepower[j], df.peakRpm[j]])
            else:
                pass
        detectedValue=[possibleMachines[0]]
        for j in range(0, len(possibleMachines),1): # finde aus den möglichen Maschinen, die nächst kleinere
            if(int(detectedValue[0][1]) >= int(possibleMachines[j][1])):
                pass
            else:
                detectedValue = [possibleMachines[j]]
        print(detectedValue)
            
        #jetzt den Update im Dataframe mit den broke & stroke von übergebenen DetectedValue (bei index = 0)
        df.horsepower[i] = detectedValue[0][2]
        df.peakRpm[i] = detectedValue[0][3]
#df.horsepower.isna().any()
#df.peakRpm.isna().any()

df[['horsepower']]=df[['horsepower']].astype('float64')
df[['peakRpm']]=df[['peakRpm']].astype('float64')

[[8, 131, '140', '5500']]
[[8, 131, '140', '5500']]


C:\Users\1810837475\.conda\envs\Kompensationsarbeit\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\1810837475\.conda\envs\Kompensationsarbeit\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# OHE
hier kann ich ein OHE verwenden

In [23]:
listOfOHE=['make','fuelType','aspiration','numOfDoors','bodyStyle','driveWheels','engineLocation',
           'engineType','numOfCylinders', 'fuelSystem']

In [24]:
from sklearn.preprocessing import OneHotEncoder
for feature in range(0,len(listOfOHE),1):
    one_hot_encoder = OneHotEncoder()
    command = 'interim = one_hot_encoder.fit_transform(df.{}.values.reshape(-1,1))'.format(listOfOHE[feature])
    exec(command)
    interim = pd.DataFrame(interim.toarray(), columns=one_hot_encoder.categories_)
    df = pd.concat([df, interim], axis=1, sort=False)
    df = df.drop([listOfOHE[feature]], axis=1)
    
'''
# show all Columns
for col in df.columns: 
    print(col)
'''

'\n# show all Columns\nfor col in df.columns: \n    print(col)\n'

In [ ]:
#df.info()

In [ ]:
df.shape

## normalized loss
Ich versuche das mit einem Modell vorherzusagen, dazu muss aber die gesammelte andere Vorverarbeitung aller anderen Feature abgeschlossen sein. => das geschieht ganz zum schluss

In [ ]:
#1) ergänzen von df um den Orginalindex, für die spätere rückordnung
indexlist=list(range(0,len(df),1))
dfIndex=pd.DataFrame(indexlist, columns=['OrginalIndex'])
df = pd.concat([dfIndex, df], axis=1, sort=False)
#split in die Werte die Nas(diese wollen wir durch das Modell predicten) haben und die die keine haben(mit diesen wollen wir modellieren)
toBePredicted = []
listForModelling = []
for i in range(0,len(df),1):
    #print(df.normalizedLosses[i])
    if(str(df.normalizedLosses[i]) == "nan"):
        interim = (list(df.iloc[i,]))
        toBePredicted.append(interim)
    else:
        interim = (list(df.iloc[i,]))
        listForModelling.append(interim)
        
titles = []
for col in df.columns: 
    titles.append(col)

In [ ]:
#Erstellen von DF spezifisch für die Anwendung modellierung bzw prediction
DFtoBePredicted = pd.DataFrame(toBePredicted, columns = titles) 
print(DFtoBePredicted.shape)
DFlistForModelling = pd.DataFrame(listForModelling, columns = titles) 
print(DFlistForModelling.shape) # <= auf dieses setzte ich jetzt ein Regressionsmodell an

In [ ]:
# Horizontale aufteilung
# über den Loop finde ich die Position von normalizedloss
for i in range(0,len(DFlistForModelling),1):
    #print(DFlistForModelling.columns[i])
    if(str(DFlistForModelling.columns[i]) == "normalizedLosses"):
        position=[i]
        break
        
input_df = DFlistForModelling.drop(['normalizedLosses'], axis=1)
print(input_df.shape)
output_df = DFlistForModelling.iloc[:,position]
print(output_df.shape)

In [ ]:
# VertikaleAufteilung in test & Train
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(
    input_df, 
    output_df,
    test_size=0.2,
    random_state = 90
)

In [ ]:
### skallieren der Daten
from sklearn.preprocessing import RobustScaler
robust_scaler_X = RobustScaler()
robust_scaler_y = RobustScaler()
robust_scaler_X.fit(train_X)
robust_scaler_y.fit(train_y)

In [ ]:
scaled_train_X = robust_scaler_X.transform(train_X)
scaled_test_X = robust_scaler_X.transform(test_X)
scaled_train_y = robust_scaler_y.transform(train_y)
scaled_test_y = robust_scaler_y.transform(test_y)

In [ ]:
# lineare Regression für die imputation
from sklearn.linear_model import LinearRegression
linerar_regressor = LinearRegression()
linerar_regressor.fit(scaled_train_X, scaled_train_y)

In [ ]:
prediction = linerar_regressor.predict(scaled_test_X)

In [ ]:
linerar_regressor.score(scaled_test_X, scaled_test_y) 

Nicht übertreiben schlecht, ich verwende nun dieses Modell um die normalized loss von dem DF vorherzusagen verwenden

In [ ]:
#Vorbereiten der 2bePredicted 

In [ ]:
prediction_df = DFtoBePredicted.drop(['normalizedLosses'], axis=1)
robust_scaler_X_toBepredicted = RobustScaler()
scaled_X_toBepredicted = robust_scaler_X.transform(prediction_df)
prediction2 = linerar_regressor.predict(scaled_X_toBepredicted)
rescaledPredictions = (robust_scaler_y.inverse_transform(prediction2))
rescaledPredictionslist = []
for i in range(0, len(rescaledPredictions),1):
    rescaledPredictionslist.append(int(rescaledPredictions[i]))


testlist = []
for i in range(0,len(df),1):
    if(str(df.normalizedLosses[i])=="nan"):
        testlist.append(str(rescaledPredictionslist[0]))
        rescaledPredictionslist.pop(0)
    else:
         testlist.append(df.normalizedLosses[i])
#create datafframe final
dfNormalizedLosses = pd.DataFrame(testlist, columns=['normalizedLosses'])
df = df.drop(['normalizedLosses', 'OrginalIndex'], axis=1)
df = pd.concat([dfNormalizedLosses, df], axis=1, sort=False)

#interim = pd.DataFrame(interim.toarray(), columns=one_hot_encoder.categories_) 

In [ ]:
df.to_csv(r'./Final.csv')

# Finaler Check
hier schau ich lediglich ob iwo noch ein NA vorhanden ist

In [ ]:
import missingno as msno
msno.matrix(df)